## Download Dataset

In [ ]:
!pip install gdown > /dev/null
!apt-get install p7zip-full -y > /dev/null

# https://drive.google.com/file/d/14FficMOD36m_IuOfpQg05qaiTjMNuKZG/view?usp=sharing
!gdown --id 14FficMOD36m_IuOfpQg05qaiTjMNuKZG
!unzip -o CAR_VIOLENCE_DATASET_final.zip > /dev/null

Downloading...
From: https://drive.google.com/uc?id=14FficMOD36m_IuOfpQg05qaiTjMNuKZG
To: /content/CAR_VIOLENCE_DATASET_final.zip
160MB [00:00, 175MB/s]


## load code

In [ ]:
!unzip -o cab_violence_detection.zip > /dev/null

## Import stuff

In [ ]:
!pip install -r skylark_autotrainer/requirements.txt > /dev/null
!pip install pytorch_lightning pytorchvideo > /dev/null
!pip install wandb > /dev/null

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torchmetrics

from pytorch_lightning import LightningModule, LightningDataModule
from pytorch_lightning.callbacks import Callback

from skylark_autotrainer import AutoTrainer, TrainerModule
from skylark_autotrainer.modules import SoftmaxCategoricalClassificationModule

import wandb

from DataModule import VideoDataModule
import models.model1 as model1
import models.model2 as model2
from VideoTrainerModule import *

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.39.0 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.1 which is incompatible.
tensorflow 2.5.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.5.0 requires tensorboard~=2.5, but you have tensorboard 2.4.1 which is incompatible.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.0 which is incompatible.
tensorflow-metadata 1.1.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.13.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
flask 1.1.4 requires click<8.0,>=5.1, but you have c

## Training configuration

In [ ]:
class SplitDataModule():
    def __init__(self, data_splits, batch_size):
      self.n = [data_split//batch_size for data_split in data_splits]

    def cal(self, train, val, test):
      return int(self.n[0] * train), int(self.n[1] * val), int(self.n[2] * test)


In [ ]:
datamodule = VideoDataModule(data_path='/content/CAR_VIOLENCE_DATASET_final',
                                 clip_duration=3.2, # 32 frames at 10 fps
                                 batch_size=16,  
                                 pin_memory=True)
print(datamodule)
split_data = SplitDataModule((812, 197, 58), batch_size = 16)

autotrainer = AutoTrainer(
    project_name = 'cab_violence_detection-Test',
    trainer_module = ViolenceDetectionModule,
    
    dataloaders = [datamodule.train_dataloader(), datamodule.val_dataloader(), datamodule.test_dataloader()],
    models = [
        {
            'model': model1.VideoModel,
            'init': {'num_classes': 2, 'lr': 8e-3, 'optimizer': 'adamax'},
            'hyperparameters': {'method': 'grid', 'lr': [1e-3, 2e-3, 4e-3],
                                'optimizer': ['adam', 'sgd', 'adamax']},
            'description': 'Pretrained r2plus1d-18 with Conv2Plus1D (~10MB)',
        },
    ],
    checkpoint = {'filename': '{epoch}-{val_acc:.4f}', 'monitor': 'val_f1', 'mode': 'max'},
    evaluation_metric = {'monitor': 'test_f1', 'mode': 'max'},
    precision = 32,
    gpus = -1,
    max_epochs = 15,
    datasets_limits = (1.0, 1.0, 1.0),
    stages = {
        'stage1': {'precision': 16, 'datasets_limits': split_data.cal(0.8, 1.0, 1.0)},
        'stage2': {'precision': 16, 'datasets_limits': split_data.cal(0.5, 0.8, 1.0)},
        'stage3': {},
    },
    restart = True, # restarting is supported now
)

Global seed set to 0


.........................................

330 videos for "1-fight"
	Training: 254
	Validation: 58
	Testing: 18
.........................................

737 videos for "0-normal"
	Training: 558
	Validation: 139
	Testing: 40
.........................................

Training videos: 812
Validation videos: 197
Testing videos: 58
.........................................



In [ ]:
autotrainer.start()

Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


  0%|          | 0.00/120M [00:00<?, ?B/s]

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



[======================================================================================================]
[================================ Initializing Stage 1 for VideoModel =================================]
[======================================================================================================]

[====================================== Performing Dev Run Test =======================================]



Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 3: val_f1 reached 0.88181 (best 0.88181), saving best model...


Validating: 0it [00:00, ?it/s]

Epoch 4, global step 504: val_f1 was not in top 1


Unfreezing layers
